In [1]:
load("ckks.py")

In [2]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Number of levels
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-3.-3.j, -1.-2.j, -1.+1.j, -2.-2.j]),
 array([ 3.+2.j, -3.-2.j,  2.+0.j,  1.+0.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[- 15032385536 - 8077064444X^4096 + 1518500250X^8192 - 7595661393X^12288 - 12884901888X^16384 - 8276468117X^20480 + 4555500750X^24576 + 1502613137X^28672 mod(2^700),
 6442450944 + 2946822291X^4096 + 16703502750X^8192 - 1502613137X^12288 + 12725903546X^20480 - 4555500750X^24576 - 7595661393X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-3.-3.j -1.-2.j -1.+1.j -2.-2.j]
[ 3.+2.00000000e+00j -3.-2.00000000e+00j  2.-7.23551663e-10j
  1.+8.94562424e-10j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.38867157e-10-1.j -4.00000000e+00-4.j  1.00000000e+00+1.j
 -1.00000000e+00-2.j]
[-2.99999999-15.j -1.         +8.j -2.         +2.j -2.         -2.j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-2.99748277-2.9980737j  -0.99799535-1.99849169j -0.9981331 +1.00069619j
 -1.99778639-1.99870603j]
